### 1. Import Libraries

In [1]:
import os
import sys
import warnings

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_openml
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

%matplotlib inline
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

### 2. Import from mlcblab

In [2]:
from mlcvlab.models.nn2 import NN2
from mlcvlab.nn.losses import l2
from mlcvlab.optim.sgd import SGD
from mlcvlab.optim.adam import Adam

### 3. Set Seed

In [3]:
np.random.seed(42)

### 4. Helper functions

In [4]:
def load_dataset():
    #TODO
    mnist = fetch_openml('mnist_784')
    x, y = mnist.data, mnist.target
    x = np.concatenate([np.ones((len(x), 1)), x], axis=1)#add additional column for bias terms
    # Convert the labels to integers
    y = y.astype(int)
    return x,y


def prepare_data(x, y):
    #TODO
    x = x / 255.0
    y = (y % 2 == 1).astype(int)
    y  = np.array(y).reshape(-1, 1)
    return x, y
    

def split_train_test(x,y):
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=10000, random_state=42)
    # Subsample a smaller portion of the data for faster training
    return X_train, X_test, y_train, y_test

def initialize_model(X_train, X_test, y_train, y_test):
    #TODO
    n_input = X_train.shape[1]
    n_output = 1
    
    # Initialize weights using Xavier method
    W0 = np.random.randn(n_input, 300) * np.sqrt(1 / n_input)
    W1 = np.random.randn(300, n_output) * np.sqrt(1 / 300)
    print(f"Size of W0 : {W0.shape}, Size of W1 : {W1.shape}")
    two_layer_nn  = NN2()
    two_layer_nn.W = [W0, W1]
    two_layer_nn.layers[0].W = W0
    two_layer_nn.layers[1].W = W1

    return two_layer_nn

def train_model(model, X_train, y_train):

    #TODO : Call SGD or Adam optimizer to train model.
    #final_W = SGD(model, X_train, y_train, lr=0.0001,R=50)
    final_W = Adam(model, X_train, y_train)
    
    return final_W[0], final_W[1]



def test_model(model, X_test, y_test, final_W):
    model.layers[0].W = final_W[0]
    model.layers[1].W = final_W[1]

    # Predict labels for test data
    y_pred = model.nn2(X_test)

    # Convert predicted probabilities to binary predictions
    y_pred = (y_pred >= 0.5).astype(int)

    # Compute accuracy
    accuracy =   np.mean(y_pred == y_test)

    return accuracy


### 5. Run the program

In [5]:
#load data
x, y = load_dataset()

#prepare data
x, y = prepare_data(x,y)

# split data set
X_train, X_test, y_train, y_test = split_train_test(x,y)

#initialize model
model = initialize_model(X_train, X_test, y_train, y_test)# return 

#training model
final_W = train_model(model, X_train, y_train)
print(f"Completed training model - final W : {final_W}")


#testing model
accuracy = test_model(model, X_test, y_test, final_W)
print(f"Completed testing model - Accuracy : {accuracy * 100:.2f}%")    

Size of W0 : (785, 300), Size of W1 : (300, 1)
Epoch 0: loss=0.24979873087716056
Epoch 10: loss=0.24981969760012016
Epoch 20: loss=0.2498092792952126
Epoch 30: loss=0.24977667401712003
Epoch 40: loss=0.24973083009391409
Epoch 50: loss=0.24968159970759457
Epoch 60: loss=0.24963748515060655
Epoch 70: loss=0.24959826891454623
Epoch 80: loss=0.24956626298335127
Epoch 90: loss=0.24954288855225049
Completed training model - final W : (array([[ 7.44927399e-07, -2.73646017e-12, -9.79551810e-10, ...,
         6.41493340e-06,  1.73022036e-06,  1.21714775e-05],
       [ 1.94965967e-09,  5.25421999e-10, -1.44563236e-09, ...,
         2.96418942e-09,  1.83647439e-10, -7.47228125e-11],
       [-1.50291267e-09,  2.57085308e-09, -2.21550062e-09, ...,
        -7.91621311e-15, -4.59368888e-09,  1.87246888e-09],
       ...,
       [ 4.54387962e-15,  7.48684965e-10,  2.36908910e-12, ...,
         5.07380488e-09, -1.26598313e-09,  1.40093580e-10],
       [ 2.47713509e-29, -1.19075511e-09,  9.25776029e-09, 